# Library Imports / Setup

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import os
import numpy as np
import random
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.0.0


In [0]:
# change current working directory
os.chdir('/content/drive/My Drive/CISC 6000: Deep Learning/Final Project (Deep Learning - Fall 2019)')
os.getcwd()

'/content/drive/My Drive/CISC 6000: Deep Learning/Final Project (Deep Learning - Fall 2019)'

In [0]:
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb, deltaE_ciede2000, deltaE_cie76
import albumentations as A
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from mpl_toolkits.axes_grid1 import ImageGrid

In [0]:
import tensorflow_datasets as tfds
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape, RepeatVector, concatenate, UpSampling2D
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model, Sequence

In [0]:
# seed for randoom number generator
def seed_everything(SEED):
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    random.seed(SEED)
seed_everything(6000)

In [0]:
BATCH_SIZE = 100
TARGET_SIZE = (224, 224)

# Define Inception ResNet V2 Model

In [0]:
#full_inres = InceptionResNetV2(weights='imagenet', include_top=True)
#inres_model = Model(full_inres.input, full_inres.layers[-2].output) # get model without softmax layer
inres_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=TARGET_SIZE+(3,), pooling='avg')
print('INRES MODEL INPUT  SHAPE:', inres_model.input.shape)
print('INRES MODEL OUTPUT SHAPE:', inres_model.output.shape)

219062272/219055592 [==============================] - 2s 0us/step
INRES MODEL INPUT  SHAPE: (None, 224, 224, 3)
INRES MODEL OUTPUT SHAPE: (None, 1536)


# Data Processing Functions

Notes:
take image
convert it first to lab

for l --> 2 * l / 100 - 1
for ab --> ab / 127
image embedding -- take lab -- concatenate it 3 times -- get predictions

plot:
gray scale image --> take input l component (X) and convert it to RGB grayscale
    lab = np.squeeze(255 * (img_l + 1) / 2)
    return color.gray2rgb(lab) / 255

output image (prediction) --> take l component and predicted ab component -- get one rgb image from this
    lab = np.empty([*img_l.shape[0:2], 3])
    lab[:, :, 0] = np.squeeze(((img_l + 1) * 50)) (l --> add 1 and then multiply 50 to this)
    lab[:, :, 1:] = img_ab * 127 (pred -- multiply back with 127)
    return color.lab2rgb(lab)

true image --> do same as above -- but combine l with original Y to get true image



In [0]:
def getImageArray(FOLDER_PATH, target_size=TARGET_SIZE, isColor=True, isSplit=True, test_split=0.1):
    img_files_lst = os.listdir(FOLDER_PATH)
    img_files_lst.sort()
    num_images = len(img_files_lst)
    x = []
    if isColor == True:
        color_mode = 'rgb'
    else:
        color_mode = 'grayscale'
    for img in img_files_lst:
        img_arr = img_to_array(load_img(path=FOLDER_PATH + '/' + img, target_size=target_size, color_mode=color_mode)) * 1.0/255
        x.append(img_arr)
    x = np.array(x)
    print('Finished converting', str(num_images), 'images as numpy arrays!')
    print('Image pixels are in range', x.min(), 'to', x.max())
    if isSplit == True:
        train, test = train_test_split(x, test_size=test_split)
        print('Train Shape:', train.shape)
        print('Test  Shape:', test.shape)
        return train, test
    return x

In [0]:
def getProcessedData(arr):
    arr_lab = rgb2lab(arr)
    X = 2 * arr_lab[:, :, :, 0] / 100 - 1 # scale to [-1, 1]
    X = np.expand_dims(X, axis=3)
    Y = arr_lab[:, :, :, 1:] / 128 # scale to [-1, 1]
    X_inres = np.concatenate([X, X, X], axis=3)
    X_inres = inres_model.predict(X_inres)
    print('Shape of X:', X.shape, 'with pixels in range', X.min(), 'to', X.max())
    print('Shape of X_inres:', X.shape, 'with pixels in range', X_inres.min(), 'to', X_inres.max())
    print('Shape of Y:', Y.shape, 'with pixels in range', Y.min(), 'to', Y.max())
    return X, X_inres, Y

In [0]:
class DataGenerator(Sequence):
    def __init__(self, datagen):
        self.datagen = datagen
        self.on_epoch_end()
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.datagen)
    
    def on_epoch_end(self):
        self.datagen.on_epoch_end()
    
    def __getitem__(self, idx):
        'Generates one batch of data'
        batch = self.datagen[idx]
        lab_batch = rgb2lab(batch)
        X = 2 * lab_batch[:, :, :, 0] / 100 - 1 # scale to [-1, 1]
        X = np.expand_dims(X, axis=3)
        Y = lab_batch[:, :, :, 1:] / 128 # scale to [-1, 1]
        X_inres = np.concatenate([X, X, X], axis=3)
        X_inres = inres_model.predict(X_inres)
        return [X, X_inres], Y

# Model Functions

In [0]:
def conv_block(x, num_filters, kernel_size, num_strides, activation, kernel_initializer, kernel_regularizer, addBN, addDR, dRate, name):
    x = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=num_strides, 
               kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, 
               padding='same', name=name)(x)
    if addBN == True:
        x = BatchNormalization(name='bn_' + name)(x)
    if addDR == True:
        x = Dropout(dRate, name='dr_' + name)(x)
    x = Activation(activation, name='act_' + name)(x)
    return x

In [0]:
def convT_block(x, num_filters, kernel_size, num_strides, activation, kernel_initializer, kernel_regularizer, addBN, addDR, dRate, name):
    x = Conv2DTranspose(filters=num_filters, kernel_size=kernel_size, strides=num_strides, 
                        kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, 
                        padding='same', name=name)(x)
    if addBN == True:
        x = BatchNormalization(name='bn_' + name)(x)
    if addDR == True:
        x = Dropout(dRate, name='dr_' + name)(x)
    x = Activation(activation, name='act_' + name)(x)
    return x

In [0]:
def buildModel(input_shape, input_shape_inres, num_filters, kernel_size, num_layers, activation, kernel_initializer, kernel_regularizer, addBN, addDR, dRate, optimizer, loss, model_name):
    inp = Input(shape=input_shape, name='input')
    x = inp
    
    # encoder
    for i in range(num_layers):
        x = conv_block(x, num_filters=num_filters, kernel_size=kernel_size, num_strides=2,
                       activation=activation, kernel_initializer=kernel_initializer, 
                       kernel_regularizer=kernel_regularizer, addBN=addBN, addDR=addDR, dRate=dRate, 
                       name='en_stridePool_' + str(num_filters))
        num_filters = num_filters * 2
        x = conv_block(x, num_filters=num_filters, kernel_size=kernel_size, num_strides=1,
                       activation=activation, kernel_initializer=kernel_initializer, 
                       kernel_regularizer=kernel_regularizer, addBN=addBN, addDR=addDR, dRate=dRate, 
                       name='en_' + str(num_filters))
    encoder_shape = K.int_shape(x)

    # inres
    inres_inp = Input(shape=input_shape_inres, name='inres_input')
    inres_out = Dense(units=int(input_shape_inres[-1] / 3), activation=activation, 
                      kernel_initializer=kernel_initializer, name='inres_dense')(inres_inp)
    inres_out = RepeatVector(encoder_shape[1] * encoder_shape[2], name='inres_repeat')(inres_out)
    inres_out = Reshape((encoder_shape[1], encoder_shape[2], int(input_shape_inres[-1] / 3)), name='inres_reshape')(inres_out)
    inres_out = concatenate([x, inres_out], axis=3, name='inres_concat')
    num_filters = int(num_filters / 2)
    x = conv_block(inres_out, num_filters=num_filters, kernel_size=1, num_strides=1,
                   activation=activation, kernel_initializer=kernel_initializer, 
                   kernel_regularizer=kernel_regularizer, addBN=addBN, addDR=addDR, dRate=dRate, 
                   name='inres_conv' + str(num_filters))
    
    # decoder
    for i in range(num_layers):
        num_filters = int(num_filters / 2)
        x = conv_block(x, num_filters=num_filters, kernel_size=kernel_size, num_strides=1,
                       activation=activation, kernel_initializer=kernel_initializer, 
                       kernel_regularizer=kernel_regularizer, addBN=addBN, addDR=addDR, dRate=dRate, 
                        name='de_' + str(num_filters))
        x = UpSampling2D(name='de_upsample' + str(num_filters))(x)
    
    out = conv_block(x, num_filters=2, kernel_size=kernel_size, num_strides=1,
                     activation='tanh', kernel_initializer=kernel_initializer, 
                     kernel_regularizer=kernel_regularizer, addBN=False, addDR=False, dRate=0.0, 
                     name='output')
    
    model = Model([inp, inres_inp], out, name=model_name)
    model.compile(optimizer=optimizer, loss=loss, metrics=['mae'])
    return model

In [0]:
def fit(model, epochs, train_gen, val_gen, verbose, callbacks_list):
    start = time.time()
    history = model.fit_generator(train_gen, epochs=epochs, steps_per_epoch=len(train_gen), verbose=verbose, 
                                  validation_data=val_gen, validation_steps=len(val_gen), callbacks=callbacks_list)
    end = time.time() - start
    print('\n\tTotal Training Time:', np.round(end, 2), 'seconds')
    print('Train Loss:', np.round(np.mean(history.history['loss']), 4), 
          '\tMAE:', np.round(np.mean(history.history['mae']), 4))
    print('Val   Loss:', np.round(np.mean(history.history['val_loss']), 4), 
          '\tMAE:', np.round(np.mean(history.history['val_mae']), 4))
    return history, model

# Evaluation Functions

In [0]:
def plotHistory(history):
    plt.figure(figsize=(20, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], 'r--', label='train_loss')
    plt.plot(history.history['val_loss'], 'b-', label='val_loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Learning Curve')
    plt.legend(loc='best')
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], 'r--', label='train_mae')
    plt.plot(history.history['val_mae'], 'b-', label='val_mae')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.title('MAE Learning Curve')
    plt.legend(loc='best')
    plt.show()

In [0]:
def predict(model, X, X_inres, Y):
    eval_results = model.evaluate([X, X_inres], Y, verbose=0)
    print('Test  Loss:', np.round(eval_results[0], 4), '\tMAE:', np.round(eval_results[1], 4))
    pred = model.predict([X, X_inres])
    canvas = np.zeros(X.shape[:-1]+(3,))
    canvas[:, :, :, 0] = (X[:, :, :, 0] + 1) * 50
    canvas[:, :, :, 1:] = pred * 128
    return canvas

In [0]:
def getSSIM(true, pred):
    return np.mean(tf.image.ssim(tf.convert_to_tensor(true, dtype='float32'), 
                                 tf.convert_to_tensor(pred, dtype='float32'), 
                                 max_val=1.0))

In [0]:
def plotResult(true, pred_arr_list, pred_label_list, num_imgs):
    for i in range(num_imgs):
        plt.figure(figsize=(20, 20))
        plt.subplot(1, len(pred_arr_list) + 2, 1)
        plt.imshow(rgb2gray(true[i]), cmap='gray')
        plt.axis('off')
        plt.title('Grayscale')
        for j in range(len(pred_arr_list)):
            plt.subplot(1, len(pred_arr_list) + 2, j+2)
            plt.imshow(lab2rgb(pred_arr_list[j][i]))
            plt.axis('off')
            plt.title(pred_label_list[j])
        plt.subplot(1, len(pred_arr_list) + 2, len(pred_arr_list) + 2)
        plt.imshow(true[i])
        plt.axis('off')
        plt.title('Ground Truth')
        plt.show()

# Final Dataset 16K - Visual Genome + Flower + Nature (224 x 224)

## Data Processing

In [0]:
data = np.load('./DATA/total-train-16K-224.npy')
print('Data Shape:', data.shape)

Data Shape: (16590, 224, 224, 3)


In [0]:
train, test = train_test_split(data, test_size=0.1)
train.shape, test.shape

((14931, 224, 224, 3), (1659, 224, 224, 3))

In [0]:
del data
gc.collect()

3

In [0]:
gc.collect()

0

In [0]:
train, val = train_test_split(train, test_size=0.2)
train.shape, val.shape

((11944, 224, 224, 3), (2987, 224, 224, 3))

In [0]:
train = train * 1.0/255
val = val * 1.0/255
test = test * 1.0/255
print('Train Shape:', train.shape)
print('Val   Shape:', val.shape)
print('Test  Shape:', test.shape)

Train Shape: (11944, 224, 224, 3)
Val   Shape: (2987, 224, 224, 3)
Test  Shape: (1659, 224, 224, 3)


In [0]:
train_datagen = ImageDataGenerator(data_format='channels_last', rotation_range=40, 
                                   shear_range=0.2, zoom_range=0.2,
                                   width_shift_range=0.2, height_shift_range=0.2, 
                                   horizontal_flip=True, vertical_flip=True)

train_generator = DataGenerator(train_datagen.flow(train, batch_size=BATCH_SIZE))
val_generator = DataGenerator(train_datagen.flow(val, batch_size=BATCH_SIZE))
print(len(train_generator), ' ', len(val_generator))

120   30


## Model Training

In [0]:
fin = buildModel(input_shape=TARGET_SIZE+(1,), input_shape_inres=(inres_model.output.shape[-1],), 
                 num_filters=64, kernel_size=3, num_layers=3, activation='relu', 
                 kernel_initializer='he_normal', kernel_regularizer=None, 
                 addBN=True, addDR=False, dRate=0.0, optimizer='adam', loss='mse', model_name='fin')
fin.summary()

Model: "fin"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
en_stridePool_64 (Conv2D)       (None, 112, 112, 64) 640         input[0][0]                      
__________________________________________________________________________________________________
bn_en_stridePool_64 (BatchNorma (None, 112, 112, 64) 256         en_stridePool_64[0][0]           
__________________________________________________________________________________________________
act_en_stridePool_64 (Activatio (None, 112, 112, 64) 0           bn_en_stridePool_64[0][0]        
________________________________________________________________________________________________

In [0]:
history, fin = fit(model=fin, epochs=200, train_gen=train_generator, val_gen=val_generator, verbose=1, 
                   callbacks_list=[ModelCheckpoint('./16K_weights.hdf5', save_best_only=False, save_weights_only=False), 
                                   ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1)])
plotHistory(history)

Epoch 1/200
120/120 [==============================] - 373s 3s/step - loss: 0.0204 - mae: 0.0420 - val_loss: 6.3961e-04 - val_mae: 0.0161
Epoch 2/200
120/120 [==============================] - 339s 3s/step - loss: 1.6787e-04 - mae: 0.0047 - val_loss: 0.0010 - val_mae: 0.0149
Epoch 3/200
120/120 [==============================] - 337s 3s/step - loss: 9.8682e-05 - mae: 0.0036 - val_loss: 9.7635e-04 - val_mae: 0.0128
Epoch 4/200
120/120 [==============================] - 336s 3s/step - loss: 9.3679e-05 - mae: 0.0037 - val_loss: 5.7361e-04 - val_mae: 0.0103
Epoch 5/200
120/120 [==============================] - 337s 3s/step - loss: 7.1216e-05 - mae: 0.0033 - val_loss: 2.1812e-04 - val_mae: 0.0070
Epoch 6/200
120/120 [==============================] - 338s 3s/step - loss: 7.0764e-05 - mae: 0.0033 - val_loss: 1.1834e-04 - val_mae: 0.0052
Epoch 7/200
120/120 [==============================] - 333s 3s/step - loss: 8.2379e-05 - mae: 0.0032 - val_loss: 7.8035e-05 - val_mae: 0.0038
Epoch 8/200
12